In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df =pd.read_csv("../input/entrepreneurial-competency-in-university-students/data.csv")
df.head(10)

In [ ]:
#checking categorical values
for i in df.columns:
    
    if df[i].dtype not in ['int64', 'float64']:
        print(f"{i} : {df[i].unique()}")

In [ ]:
encode_dummies = ['IndividualProject', 'Gender', 'City','Influenced', 'MentalDisorder']
encode_label = ['EducationSector', 'KeyTraits']
df_new = pd.get_dummies(df, columns = encode_dummies)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()    
df_new['EducationSector'] = le.fit_transform(df_new['EducationSector'])
df_new['KeyTraits'] = le.fit_transform(df_new['KeyTraits'])

In [ ]:
df_new.head()

In [ ]:
df_new['EducationSector'].value_counts()

In [ ]:
df_new.info()

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
stopwords = set(stopwords.words('english'))
import string

df2 = df_new[['ReasonsForLack']]
df2 = df2.dropna()

print(stopwords)

In [ ]:
df2['ReasonsForLack'] = df2['ReasonsForLack'].str.lower().str.split()
df2['ReasonsForLack'] =df2['ReasonsForLack'].apply(lambda x: [item for item in x if item not in stopwords])
df2['ReasonsForLack'] = df2['ReasonsForLack'].apply(lambda x : [item.translate(str.maketrans('', '', string.punctuation)) for item in x])

In [ ]:
df2['ReasonsForLack']

In [ ]:
df2explode=pd.DataFrame(df2['ReasonsForLack'].explode())
df2explode

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
df2explode.value_counts()

In [ ]:
df2wordcount=pd.DataFrame(df2explode.groupby('ReasonsForLack')['ReasonsForLack'].count().sort_values(ascending=False))


In [ ]:
df2wordcount=df2wordcount.rename(columns={'ReasonsForLack': 'count'})
df2wordcount.index = df2wordcount.index.rename('Word')

In [ ]:
df2wordcount =df2wordcount.reset_index()
df2wordcount

In [ ]:

df2wordcount.plot.bar(x='Word')

In [ ]:
import seaborn as sns
sns.barplot(y = 'Word', x = 'count', data = df2wordcount)

In [ ]:
df_new.columns

In [ ]:
df_new = df_new.drop("ReasonsForLack", axis = 1)

In [ ]:
df_new.corr().abs()['y'].sort_values(ascending = False)

In [ ]:
df_new['y'].value_counts()

In [ ]:
zero  = df_new[df_new['y']==0]
one = df_new[df_new['y']==1]
from sklearn.utils import resample
df_minority_upsampled = resample(one, replace = True, n_samples = 128)
df_new = pd.concat([zero, df_minority_upsampled])

from sklearn.utils import shuffle
df_new = shuffle(df_new)

In [ ]:
df_new['y'].value_counts()

In [ ]:
X = df_new.drop(['y'], axis =1)
Y = df_new['y']

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [ ]:
model = XGBClassifier(learning_rate =0.9, n_estimators = 1500,seed=27)
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [ ]:
accuracy_score(y_test,pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))